In [8]:
import sys  
sys.path.insert(0, '../../pycurvis')
from data.vec_io import *
from data.vis_io import *
import vtk
from vtkmodules.util import numpy_support

In [19]:
sgr = vtk.vtkStructuredGridReader()
sgr.SetFileName("data/mountain_backcurve40_20000.vts")
sgr.Update()
sg = sgr.GetOutput()

In [24]:
sg.GetNumberOfPoints()

0

In [26]:
phys = read_vec("data/mantle_phys.vec")

In [28]:
phys.shape

(201, 360, 140, 3)